In [2]:
from pydantic import BaseModel, Field, field_validator, model_validator, AliasChoices

In [ ]:
from pydantic import BaseModel, Field

class ProductSubmission(BaseModel):
    name: str = Field(..., min_length=2, max_length=50, description="Product name")
    price: float = Field(gt=0, lt=10000, description="Product price in USD")
    sku: str = Field(regex=r"^[A-Z]{2}\d{6}$", description="Stock keeping unit code")
    stock: int = Field(ge=0, default=0, description="Available inventory")
    tags: list[str] = Field(default_factory=list)

# Validates all constraints
product = ProductSubmission(name="Laptop", price=999.99, sku="AB123456", stock=10)

In [ ]:
from pydantic import BaseModel, field_validator

class Registration(BaseModel):
    username: str
    password: str
    password_confirm: str
    
    @field_validator('username', mode="before")
    @classmethod
    def username_alphanumeric(cls, v: str) -> str:
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v
    
    @field_validator('password_confirm')
    @classmethod
    def passwords_match(cls, v: str, info) -> str:
        if v != info.data.get('password'):
            raise ValueError('Passwords do not match')
        return v

# Example usage
user = Registration(username="john123", password="secret", password_confirm="secret")

In [ ]:
from pydantic import BaseModel, model_validator
from datetime import date

class TravelBooking(BaseModel):
    departure_date: date
    return_date: date
    is_one_way: bool = False
    
    @model_validator(mode='after')
    def check_return_date(self) -> 'TravelBooking':
        if not self.is_one_way and self.return_date <= self.departure_date:
            raise ValueError('Return date must be after departure date for round trips')
        if self.is_one_way and self.return_date is not None:
            # Normalize the data by removing unneeded return date
            self.return_date = None
        return self

# This will fail validation
try:
    booking = TravelBooking(
        departure_date=date(2023, 5, 1),
        return_date=date(2023, 4, 30),
        is_one_way=False
    )
except ValueError as e:
    print(f"Error: {e}")

In [4]:
from pydantic import BaseModel, AliasChoices, Field

class ApiResponse(BaseModel):
    user_id: int = Field(validation_alias=AliasChoices('userId', 'user_id', 'id'))
    full_name: str = Field(validation_alias=AliasChoices('fullName', 'full_name', 'name'))

# Works with any of the alias options
response1 = ApiResponse(userId=123, fullName="John Doe")
response2 = ApiResponse(user_id=456, full_name="Jane Smith")
response3 = ApiResponse(id=789, name="Alex Johnson")

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from geopy.distance import geodesic  # Optional, if you want distance calculations

# Location Model
class Coordinates(BaseModel):
    latitude: float = Field(..., ge=-90, le=90)
    longitude: float = Field(..., ge=-180, le=180)

# Player Model
class Player(BaseModel):
    name: str
    location: Coordinates

# Team Model
class Team(BaseModel):
    name: str
    location: Coordinates
    players: List[Player]


In [ ]:
team = Team(
    name="Global Hawks",
    location={"latitude": 40.7128, "longitude": -74.0060},
    players=[
        {"name": "Alice", "location": {"latitude": 34.0522, "longitude": -118.2437}},
        {"name": "Bob", "location": {"latitude": 51.5074, "longitude": -0.1278}},
    ]
)

print(team.json(indent=2))


In [ ]:
def distance_to_team(player: Player, team: Team) -> float:
    return geodesic(
        (player.location.latitude, player.location.longitude),
        (team.location.latitude, team.location.longitude)
    ).kilometers

# Example
for p in team.players:
    print(f"{p.name} is {distance_to_team(p, team):.2f} km from the team HQ")


In [ ]:
player = Player(name="Alex", age=25)

# To JSON
player.json()

# To dict
player.dict()


In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Coordinates(BaseModel):
    latitude: float = Field(..., ge=-90, le=90)
    longitude: float = Field(..., ge=-180, le=180)

class Player(BaseModel):
    name: str
    location: Coordinates

class Team(BaseModel):
    name: str
    location: Coordinates
    players: List[Player]


In [ ]:
import pandas as pd
from collections import defaultdict

# Load data
teams_df = pd.read_excel("teams.xlsx", sheet_name="Teams")
players_df = pd.read_excel("teams.xlsx", sheet_name="Players")

# Group players by team
team_players = defaultdict(list)

for _, row in players_df.iterrows():
    player = Player(
        name=row['player_name'],
        location=Coordinates(latitude=row['latitude'], longitude=row['longitude'])
    )
    team_players[row['team_name']].append(player)

# Create Team objects
team_models = []

for _, row in teams_df.iterrows():
    team = Team(
        name=row['team_name'],
        location=Coordinates(latitude=row['latitude'], longitude=row['longitude']),
        players=team_players[row['team_name']]
    )
    team_models.append(team)


In [ ]:
import json

# Save to file
with open("teams_data.json", "w") as f:
    json.dump([team.dict() for team in team_models], f, indent=2)


In [ ]:
with open("teams_data.json") as f:
    raw = json.load(f)
    loaded_teams = [Team(**t) for t in raw]
